Module 3 Assignment - Part 4: Use TF/Keras to Create both an RNN and an LSTM and Compare Results
---

For this part, you can choose any dataset you like. I recommend choosing text data (such as movie reviews: <https://keras.io/api/datasets/imdb/>) and predicting sentiment. However, I will allow you to select any sequential dataset you like as long as it is appropriate for RNNs.

Use TF/Keras to create two models. The first will be a simple RNN. The second will be a LSTM model.

There are great examples online that you can review and learn from (but not copy :)). You can also have a look at one of my examples that compares ANN, CNN, RNN, and LSTM: <https://gatesboltonanalytics.com/?page_id=903>

Just as in Parts 1 and 2 (I will not repeat the requirements again here - but please assume them) you will illustrate your process and results. You will also discuss and illustrate a comparison between the RNN and the LSTM models.

---


In [1]:
#libraries
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
import tensorflow.keras
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, LSTM
from tensorflow.keras.optimizers import Adam
import matplotlib.pyplot as plt
from tensorflow.keras import layers
import numpy as np
import pandas as pd
import seaborn as sns

2023-11-29 00:44:05.277574: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-11-29 00:44:05.456972: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


# Data

(Using IMBD movie review data.)

In [3]:
Data = tf.keras.datasets.imdb
(X_train, y_train),(X_test, y_test) = Data.load_data()

17464789/17464789 [==============================] - 0s 0us/step
